In [1]:
%reload_ext autoreload
%autoreload 2

## CellLogger subsystem

This test tests the CellLogger subsystem, accessible via `exp.cl` once one of the `IPyExperiments` subclasses has been initiated.

## Test specifics

Since we need to validate the the output, we have to capture it first. The way jupyter is setup, is that in once cell you set up a capture with `%%capture` magick and then in the next cell you can analyze it. That's why each test group has two cells, the first one doing the action to be tested and the following one doing the validatations.

Moreover, the output of this test becomes confusing because the capture mechanism somehow messes things up which leads to re-running the `post_run_cell` callback of the CellLogger subsystem again - as a result you get a bogus output with 0's regardless of the code being run. It doesn't interfere with the testing, but it does interfere with things like `.data` which gets reset because of that, showing invalid information - therefore we can only test `.data` w/o capturing the cell's output.

## Setup

In [2]:
from ipyexperiments import *
from utils.text import *
import re

## Consume

In [3]:
#if 'exp' in locals(): exp.cl.stop() # helps debug
exp1 = IPyExperimentsPytorch(exp_enable=False)
exp1.cl


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)



･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.005
･ CPU:         0       0     2146 MB |
･ GPU:         0       0     3689 MB |


In [4]:
%%capture output
cpu1 = consume_cpu_ram_128mb()
gpu1 = consume_gpu_ram_256mb()

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.127
･ CPU:         0       0     2275 MB |
･ GPU:         0       0     3945 MB |


In [5]:
"""test_report"""
output = str(output)
print_output(output)

check_report_strings(output)
check_report_cpu(output, consumed_expected=128, peaked_expected=0, abs_tol=2)
check_report_gpu(output, consumed_expected=256, peaked_expected=0, abs_tol=0)

# cleanup
del cpu1, gpu1

'test_report'

Captured output:
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.116
| ･ CPU:       128       0     2275 MB |
| ･ GPU:       256       0     3945 MB |
| 

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.006
･ CPU:         0       0     2147 MB |
･ GPU:      -256     256     3689 MB |


## Consume/release leading to positive peak numbers

In [6]:
%%capture output
# test peak measurement
# here we consume 256MB of RAM and release 128MB 
# testing: Consumed 128, Peaked 128
cpu1 = consume_cpu_ram_128mb()
cpu2 = consume_cpu_ram_128mb()
del cpu1

# here we consume 512MB of RAM and release 256MB
# testing: Consumed 256, Peaked 256
gpu1 = consume_gpu_ram_256mb()
gpu2 = consume_gpu_ram_256mb()
del gpu1


･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.246
･ CPU:         0       0     2275 MB |
･ GPU:         0     256     3945 MB |


In [7]:
"""test_peak_memory_usage"""
output = str(output)
print_output(output)

check_report_cpu(output, consumed_expected=128, peaked_expected=128, abs_tol=2)
check_report_gpu(output, consumed_expected=256, peaked_expected=256, abs_tol=2)

# cleanup
del cpu2, gpu2

'test_peak_memory_usage'

Captured output:
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.235
| ･ CPU:       128     128     2275 MB |
| ･ GPU:       256     256     3945 MB |
| 

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.006
･ CPU:         0       0     2147 MB |
･ GPU:      -256     256     3689 MB |


## .data accessor validation

In [8]:
# no capture! breaks .data since it re-runs the post_run_cell, again, resetting .data
# here we consume 256MB of RAM and release 128MB - so that we can test peak measurement
# testing: Consumed 128, Peaked 128
cpu1 = consume_cpu_ram_128mb()
cpu2 = consume_cpu_ram_128mb()
del cpu1

# here we consume 512MB of RAM and release 256MB - so that we can test peak measurement
# testing: Consumed 256, Peaked 256
gpu1 = consume_gpu_ram_256mb()
gpu2 = consume_gpu_ram_256mb()
## Consume/Release Positive Peak
del gpu1


･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.263
･ CPU:       128     128     2275 MB |
･ GPU:       256     256     3945 MB |


In [9]:
"""test_data_accessor"""
cpu_mem   = exp1.cl.data.cpu
gpu_mem   = exp1.cl.data.gpu
time_data = exp1.cl.data.time
check_match(consumed_reported=b2mb(cpu_mem.used_delta), peaked_reported=b2mb(cpu_mem.peaked_delta), 
            consumed_expected=128,                      peaked_expected=128,  abs_tol=1)
check_match(consumed_reported=b2mb(gpu_mem.used_delta), peaked_reported=b2mb(gpu_mem.peaked_delta), 
            consumed_expected=256,                      peaked_expected=256, abs_tol=1)

# cleanup
del cpu2, gpu2

'test_data_accessor'

134224187
134232116
268435456
268435456
･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.006
･ CPU:         0       0     2147 MB |
･ GPU:      -256     256     3689 MB |


## .stop

In [10]:
"""test_stop"""
exp1.cl.stop()
#check that no output appears after this one

'test_stop'

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.051
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |


In [11]:
%%capture output
cpu1 = consume_cpu_ram_128mb()

In [12]:
"""test_report"""
output = str(output)
print_output(output)
assert output == "", "there should be no output as logger has been stopped"

# cleanup
del cpu1
del exp1

'test_report'

No captured output


## Implicit destroy

In [13]:
locals_unset(['exp10'])    

In [14]:
%%capture output
# test destroy which happens when obj is redefined 
# this one tests with the exp-system turned on, cl-system turned on
# this is a pretty normal situation, considering that someone will be reloading the same cell
# do not change this test!
exp10 = IPyExperimentsPytorch(exp_enable=True, cl_enable=True)
exp10 = IPyExperimentsPytorch(exp_enable=True, cl_enable=True)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.032
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |


In [15]:
output = str(output)
print_output(output)
assert "Error" not in output, "shouldn't fail on auto-destruction"

match = re.findall(r'started', output)
assert len(match) == 2, f"should have started twice, got {len(match)}"

match = re.findall(r'Finishing', output)
assert len(match) == 1, f"should have finished once, got {len(match)}"

Captured output:
| 
| *** Experiment started with the Pytorch backend
| Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:  2147  2557  31588 MB   6.80% 
| GPU:  3689  4430   8119 MB  45.43% 
| 
| 
| 
| *** Experiment started with the Pytorch backend
| Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:  2147  2557  31588 MB   6.80% 
| GPU:  3689  4430   8119 MB  45.43% 
| 
| 
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.024
| ･ CPU:         0       0     2147 MB |
| ･ GPU:         0       0     3689 MB |
| 
| IPyExperimentsPytorch: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:       0       0 MB (100.00%)
| GPU:       0       0 MB (100.00%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:  2147  2557  31588 MB   6.80% 
| GPU:  3689  4

In [16]:
# cleanup
del exp10

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.078
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |
･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.078
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:

## Implicit destroy #2

In [17]:
%%capture output
# test destroy which happens when obj is redefined 
# this one tests with the exp-system turned off, cl-system turned on
# this is a pretty normal situation, considering that someone will be reloading the same cell
# do not change this test!
exp11 = IPyExperimentsPytorch(exp_enable=False, cl_enable=True)
exp11 = IPyExperimentsPytorch(exp_enable=False, cl_enable=True)


Deleted: match

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:       0       0 MB (100.00%)
GPU:       0       0 MB (100.00%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:  2147  2551  31588 MB   6.80% 
GPU:  3689  4430   8119 MB  45.43% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.020
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |


In [18]:
output = str(output)
print_output(output)
assert "Error" not in output, "shouldn't fail on auto-destruction"

Captured output:
| 
| *** Experiment started with the Pytorch backend
| Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)
| 
| 
| *** Experiment started with the Pytorch backend
| Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)
| 
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.022
| ･ CPU:         0       0     2147 MB |
| ･ GPU:         0       0     3689 MB |
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.010
| ･ CPU:         0       0     2147 MB |
| ･ GPU:         0       0     3689 MB |
| 

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.000
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |


In [ ]:
# cleanup
del exp11

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.043
･ CPU:         0       0     2147 MB |
･ GPU:         0       0     3689 MB |


In [ ]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()